In [1]:
#Import the dependicies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
#Create a set of random latitude and longitude combinations
lats = np.random.uniform(low = -90.000, high = 90.000, size = 1500)
lngs = np.random.uniform(low = -180.000, high = 1800, size = 1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [5]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)